```
--- Day 22: Monkey Market ---
As you're all teleported deep into the jungle, a monkey steals The Historians' device! You'll need get it back while The Historians are looking for the Chief.

The monkey that stole the device seems willing to trade it, but only in exchange for an absurd number of bananas. Your only option is to buy bananas on the Monkey Exchange Market.

You aren't sure how the Monkey Exchange Market works, but one of The Historians senses trouble and comes over to help. Apparently, they've been studying these monkeys for a while and have deciphered their secrets.

Today, the Market is full of monkeys buying good hiding spots. Fortunately, because of the time you recently spent in this jungle, you know lots of good hiding spots you can sell! If you sell enough hiding spots, you should be able to get enough bananas to buy the device back.

On the Market, the buyers seem to use random prices, but their prices are actually only pseudorandom! If you know the secret of how they pick their prices, you can wait for the perfect time to sell.

The part about secrets is literal, the Historian explains. Each buyer produces a pseudorandom sequence of secret numbers where each secret is derived from the previous.

In particular, each buyer's secret number evolves into the next secret number in the sequence via the following process:

Calculate the result of multiplying the secret number by 64. Then, mix this result into the secret number. Finally, prune the secret number.
Calculate the result of dividing the secret number by 32. Round the result down to the nearest integer. Then, mix this result into the secret number. Finally, prune the secret number.
Calculate the result of multiplying the secret number by 2048. Then, mix this result into the secret number. Finally, prune the secret number.
Each step of the above process involves mixing and pruning:

To mix a value into the secret number, calculate the bitwise XOR of the given value and the secret number. Then, the secret number becomes the result of that operation. (If the secret number is 42 and you were to mix 15 into the secret number, the secret number would become 37.)
To prune the secret number, calculate the value of the secret number modulo 16777216. Then, the secret number becomes the result of that operation. (If the secret number is 100000000 and you were to prune the secret number, the secret number would become 16113920.)
After this process completes, the buyer is left with the next secret number in the sequence. The buyer can repeat this process as many times as necessary to produce more secret numbers.

```

In [1]:
# Function to calculate the next secret number in the sequence
def generate_next_secret(secret_number):
    # Step 1: Multiply by 64, mix, prune
    secret_number ^= (secret_number * 64) % 16777216
    secret_number %= 16777216

    # Step 2: Divide by 32 (integer division), mix, prune
    secret_number ^= (secret_number // 32) % 16777216
    secret_number %= 16777216

    # Step 3: Multiply by 2048, mix, prune
    secret_number ^= (secret_number * 2048) % 16777216
    secret_number %= 16777216

    return secret_number

# Function to simulate the sequence and get the 2000th secret number
def simulate_buyer(initial_secret):
    secret = initial_secret
    for _ in range(2000):
        secret = generate_next_secret(secret)
    return secret

# Main function to read input and calculate the result
def main():
    # Read input from file
    with open("input.txt", "r") as file:
        initial_secrets = [int(line.strip()) for line in file.readlines()]

    # Calculate the 2000th secret number for each buyer
    results = [simulate_buyer(secret) for secret in initial_secrets]

    # Calculate the total sum of the 2000th secret numbers
    total_sum = sum(results)

    # Print the result
    print("Sum of the 2000th secret numbers:", total_sum)

# Run the main function
if __name__ == "__main__":
    main()


Sum of the 2000th secret numbers: 20506453102


```
--- Part Two ---
Of course, the secret numbers aren't the prices each buyer is offering! That would be ridiculous. Instead, the prices the buyer offers are just the ones digit of each of their secret numbers.


```

In [3]:
def generate_next_secret(secret_number):
    secret_number ^= (secret_number * 64) % 16777216
    secret_number %= 16777216
    secret_number ^= (secret_number // 32) % 16777216
    secret_number %= 16777216
    secret_number ^= (secret_number * 2048) % 16777216
    secret_number %= 16777216
    return secret_number

def get_price_sequence(initial_secret):
    # Generate all prices at once
    prices = []
    secret = initial_secret
    for _ in range(2001):  # We need 2001 to get 2000 changes
        prices.append(secret % 10)
        secret = generate_next_secret(secret)
    return prices

def find_sequences(prices):
    # Create a dictionary to store where each sequence appears and its corresponding price
    sequences = {}
    changes = []

    # Calculate all changes
    for i in range(1, len(prices)):
        changes.append(prices[i] - prices[i-1])

    # For each position, record the 4-change sequence that starts there
    for i in range(len(changes) - 3):
        seq = tuple(changes[i:i+4])  # Convert to tuple so it can be used as dict key
        if seq not in sequences:  # Only keep the first occurrence
            sequences[seq] = prices[i+4]

    return sequences

def main():
    with open("input.txt", "r") as file:
        initial_secrets = [int(line.strip()) for line in file.readlines()]

    # Pre-calculate all sequences for each buyer
    buyer_sequences = []
    for secret in initial_secrets:
        prices = get_price_sequence(secret)
        sequences = find_sequences(prices)
        buyer_sequences.append(sequences)

    # Find the sequence that appears in most buyers with highest total
    best_total = 0
    best_sequence = None

    # Get all unique sequences that appear in any buyer's data
    all_sequences = set()
    for sequences in buyer_sequences:
        all_sequences.update(sequences.keys())

    # Check each sequence that actually appears in the data
    for seq in all_sequences:
        total = sum(sequences.get(seq, 0) for sequences in buyer_sequences)
        if total > best_total:
            best_total = total
            best_sequence = seq

    print(f"Best sequence: {list(best_sequence)}")
    print(f"Maximum bananas: {best_total}")

if __name__ == "__main__":
    main()

Best sequence: [0, 0, -1, 2]
Maximum bananas: 2423
